In [2]:
import pandas as pd
df= pd.read_excel('CHGenesOrdenadosConVecinos.xlsx',engine='openpyxl',skiprows=[0],usecols=range(8))
df.head()

,Gen Abrev,Category,Name of the gene,Inicio,Final,RNASeq,Microarray,SAGE
0,TRNAS-GGA,NaN,NaN,32091689,32091761,NaN,NaN,NaN
1,LOC102173449,NaN,NaN,32837131,32838076,NaN,NaN,NaN
2,LOC102169333,NaN,NaN,33089112,33090132,NaN,NaN,NaN
3,LOC108636757,NaN,NaN,33764252,33768221,NaN,NaN,NaN
4,LOC102187712,NaN,NaN,33810174,33811741,NaN,NaN,NaN


In [3]:
gen=df['Gen Abrev']
gen=gen.tolist()
gen[0:10]

['TRNAS-GGA',
 'LOC102173449',
 'LOC102169333',
 'LOC108636757',
 'LOC102187712',
 'VGLL3',
 'CHMP2B',
 'POU1F1',
 'HTR1F',
 'LOC102188976']

In [4]:
links=[]
for i in gen:
    url = 'https://www.genecards.org/cgi-bin/carddisp.pl?gene='+i+'#expression'
    #print(url)
    links.append(url)
print(links[8])

https://www.genecards.org/cgi-bin/carddisp.pl?gene=HTR1F#expression


In [7]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import re 
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
link='https://www.genecards.org/cgi-bin/carddisp.pl?gene=HTR1F#expression'
# se simula el uso de chrome
driver = webdriver.Chrome(executable_path='C:/Users/Sat/Downloads/chromedriver_win32/chromedriver.exe')
html=[]
for link in links:
    driver.get(link)#se abre el link
    # Se busca el elemento de la imagen
    try:
        element = driver.find_element_by_id("expressionImage")
        soup = BeautifulSoup(driver.page_source, features='html.parser')
        while(soup.find("area")==None):
            driver.get(link)
            soup = BeautifulSoup(driver.page_source, features='html.parser')
        html.append(soup)
    except:
        html.append(None)
        continue
driver.quit()

WebDriverException: Message: 'chromedriver.exe' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [7]:
genes1=[]
for gen1 in html:
    g=[]
    if gen1 == None:
        g.append(None)
    else:
        soup=gen1
        g.append(soup.find("span",{"class":"gc-category"}).text)
        g.append(soup.find_all("span",{"class":"aliasMainName"})[1].text)
        lista=soup.find_all("area",{"title":re.compile('.*Breast.*')})
        g.append(lista[0]["title"].split(",")[0]+', '+lista[3]["title"].split(",")[0])
        g.append(lista[1]["title"].split(",")[0])
        g.append(lista[2]["title"].split(",")[0])
    genes1.append(g)
print(genes1)

[[None], [None], [None], [None], [None], ['(Protein Coding)', 'Vestigial Like Family Member 3', '25, 44', 'Breast - No Data', '0'], ['(Protein Coding)', 'Charged Multivesicular Body Protein 2B', '42, 44', 'Breast - No Data', '21'], ['(Protein Coding)', 'POU Class 1 Homeobox 1', '2, 0', 'Breast - No Data', '3'], ['(Protein Coding)', '5-Hydroxytryptamine Receptor 1F', '6, 3', 'Breast - No Data', 'Breast - No Data'], [None], ['(Protein Coding)', 'CGG Triplet Repeat Binding Protein 1', '43, 49', 'Breast - No Data', '18'], ['(Protein Coding)', 'Zinc Finger Protein 654', '23, 20', 'Breast - No Data', '0'], [None], [None], ['(Protein Coding)', 'Solute Carrier Family 12 Member 8', '10, 11', 'Breast - No Data', 'Breast - No Data'], [None], ['(Protein Coding)', 'Zinc Finger Protein 148', '22, 26', 'Breast - No Data', '3'], ['(Protein Coding)', 'Sorting Nexin 4', '35, 38', 'Breast - No Data', '6'], ['(Protein Coding)', 'Oxysterol Binding Protein Like 11', '22, 27', 'Breast - No Data', 'Breast - N

In [24]:
import numpy as np
import pandas as pd
genes2=pd.DataFrame(genes1, columns=['Category','Name of the gene','RNASeq','Microarray','SAGE'])
t1=df['Gen Abrev']
t2=genes2.iloc[:,[0,1]]
t3=df[['Inicio','Final']]
t4=genes2.iloc[:,2:5]
H = pd.concat([t1,t2,t3,t4], axis=1)
H = H.fillna(value='NaN')
H.head(10)

,Gen Abrev,Category,Name of the gene,Inicio,Final,RNASeq,Microarray,SAGE
0,TRNAS-GGA,NaN,NaN,32091689,32091761,NaN,NaN,NaN
1,LOC102173449,NaN,NaN,32837131,32838076,NaN,NaN,NaN
2,LOC102169333,NaN,NaN,33089112,33090132,NaN,NaN,NaN
3,LOC108636757,NaN,NaN,33764252,33768221,NaN,NaN,NaN
4,LOC102187712,NaN,NaN,33810174,33811741,NaN,NaN,NaN
5,VGLL3,(Protein Coding),Vestigial Like Family Member 3,33882149,33934641,"25, 44",Breast - No Data,0
6,CHMP2B,(Protein Coding),Charged Multivesicular Body Protein 2B,34199383,34231630,"42, 44",Breast - No Data,21
7,POU1F1,(Protein Coding),POU Class 1 Homeobox 1,34235896,34251973,"2, 0",Breast - No Data,3
8,HTR1F,(Protein Coding),5-Hydroxytryptamine Receptor 1F,34811253,34812505,"6, 3",Breast - No Data,Breast - No Data
9,LOC102188976,NaN,NaN,34826096,34855603,NaN,NaN,NaN


In [17]:
H.to_excel("CHGenesOrdenadosConVecinos1.xlsx",index=False)